In [1]:
!nvidia-smi

Wed Aug  2 12:51:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:31:00.0 Off |                    0 |
| N/A   27C    P0    48W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pickle
import os

seed = 2023

In [3]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'cuda'

In [4]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [5]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [6]:
output_dir = "therapist_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [7]:
%run ../load_anno_mi.py

In [8]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-02 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-02 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-02 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-02 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-02 00:00:34


In [9]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: Fine-tune BERT for classification

In [10]:
num_epochs = 10
seeds = [1, 12, 123]
validation_metric = "f1"

In [11]:
bert_classifier = fine_tune_transformer_average_seed(num_epochs=num_epochs,
                                                     pretrained_model_name="bert-base-uncased",
                                                     df=anno_mi,
                                                     feature_name="utterance_text",
                                                     label_column="main_therapist_behaviour",
                                                     seeds=seeds,
                                                     path_indices=therapist_index,
                                                     k_fold=True,
                                                     validation_metric=validation_metric,
                                                     results_output=f"{output_dir}/bert_classifier.csv",
                                                     device=device,
                                                     verbose=False)

[INFO] Setting return_special_tokens_mask=True
[INFO] Tokenizing the dataset...


Map:   0%|          | 0/6826 [00:00<?, ? examples/s]

[INFO] Saving the tokenized text for each sentence into `.df['tokens']`...


Map:   0%|          | 0/6826 [00:00<?, ? examples/s]

[INFO] Creating tokenized dataframe and setting in `.tokenized_df` attribute...
[INFO] Note: 'text_id' is the column name for denoting the corresponding text id
[INFO] Setting up TrainingArguments object and saving to `.training_args`.
[INFO] Setting up Trainer object, and saving to `.trainer`.
[INFO] Training model with 109485316 parameters...


...

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.614503,0.785912,0.748559
2,No log,0.517902,0.820300,0.792566
3,No log,0.536980,0.815862,0.791296
4,No log,0.588621,0.812535,0.790754
5,No log,0.626598,0.810316,0.789792
6,No log,0.694494,0.804215,0.784853
7,No log,0.657896,0.803661,0.783592
8,No log,0.664794,0.815308,0.796593
9,0.323200,0.705520,0.808098,0.786182
10,0.323200,0.694661,0.808652,0.788637


[INFO] Training completed!


  0%|          | 0/1365 [00:00<?, ?it/s]

saving the results dataframe to CSV in therapist_talk_type_output/bert_classifier.csv


In [12]:
bert_classifier["f1"].mean()

0.8000210425016903

In [13]:
bert_classifier["precision"].mean()

0.8023882427544045

In [14]:
bert_classifier["recall"].mean()

0.7992662902226257

In [15]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.86419294, 0.68140363, 0.7774351 , 0.87705251])

In [16]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.84401846, 0.69953113, 0.76277041, 0.90323298])

In [17]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.88587513, 0.66561265, 0.79312755, 0.85244984])